In [1]:
# ETL --> EXTRACT HDF5 files into Spark dataframe
from pyspark.sql import SparkSession, Row
from pyspark import SparkContext
from pyspark.sql.types import *
import pandas as pd
import tables
import h5py

# Get list of all HDF5 files contained in the folder 
def build_list_file(root_folder):
  files = []
  if(LIMIT_SONG != -1):
    list_patch = dbutils.fs.ls(root_folder)[:LIMIT_SONG]
  else:
    list_patch = dbutils.fs.ls(root_folder)
  
  #Fetch file names
  for file in list_patch:
    files.append("/dbfs//FileStore/tables/songs/" + file[1])
  return files
    
def read_hdf5(path):
  # Fetch table objects through HDFStore
  store = pd.HDFStore(path, mode = 'r')
  metadata = store.get("metadata/songs")
  analysis = store.get("analysis/songs")
  year = store.get("musicbrainz/songs")

  # Fetch arrays through h5py metadata (+ encode strings which were encoded in an old format)
  h5 = h5py.File(path, 'r')
  artist_terms = pd.Series([[item.decode() for item in list(h5.get("metadata/artist_terms"))]], name = "artist_terms")
  similar_artists = pd.Series([[item.decode() for item in list(h5.get("metadata/similar_artists"))]], name = "similar_artists")
  
  # Fetch arrays by h5py - analysis (+ turn numpy numbers into float)
  bars_start = pd.Series([[float(item) for item in list(h5.get("/analysis/bars_start"))]], name="bars_start")
  beats_start = pd.Series([[float(item) for item in list(h5.get("/analysis/beats_start"))]], name="beats_start")
  sections_start =pd.Series([[float(item) for item in list(h5.get("/analysis/sections_start"))]], name="sections_start")
  segments_start = pd.Series([[float(item) for item in list(h5.get("/analysis/segments_start"))]], name="segments_start")
  tatums_start = pd.Series([[float(item) for item in list(h5.get("/analysis/tatums_start"))]], name="tatums_start")
  
  # THE FOLLOWING FIELDS HAVE BEEN EXCLUDED FROM THE ORIGINAL SONG FEATURES
  # EXCLUDED (confidence arrays): bars_confidence, beats_confidence, sections_confidence, segments_confidence, tatums_confidence 
  # EXCLUDED (segments metadata): segments_pitches, segments_timbre, segments_loudness_max, segments_loudness_max_time, segments_loudness_start
  # EXCLUDED (artist terms weights and frequency): artist_terms_freq, artist_terms_weight,
  
  # Merge outcomes
  song = pd.concat([metadata, analysis,year, artist_terms, similar_artists, bars_start,beats_start, sections_start, segments_start, tatums_start], axis=1, join='outer')
  
  # Delete some columns 
  song = song.drop(columns = ["idx_artist_terms", "idx_similar_artists", "audio_md5", "idx_bars_confidence","idx_bars_start", "idx_beats_confidence", "idx_beats_start", "idx_sections_confidence", "idx_sections_start", "idx_segments_confidence", "idx_segments_loudness_max", "idx_segments_loudness_max_time", "idx_segments_loudness_start",	"idx_segments_pitches", "idx_segments_start", "idx_segments_timbre", "idx_tatums_confidence", "idx_tatums_start", "time_signature_confidence", "analyzer_version", "key_confidence", "mode_confidence", "idx_artist_mbtags", "track_7digitalid", "release_7digitalid", "artist_playmeid","artist_mbid", "artist_7digitalid","analysis_sample_rate"])

  return song.values.tolist()

#-----------------------------------------------------------------------------------
ROOT_FOLDER = "/FileStore/tables/songs"
LIMIT_SONG = -1 #All songs

# GET: Spark context
spark = SparkSession.builder.appName("Milion Songs Dataset").getOrCreate()
sc = SparkContext.getOrCreate() 

# READ: HDF5 files (using RDD)
files_list = build_list_file(ROOT_FOLDER)
rdd = sc.parallelize(files_list).flatMap(lambda path: read_hdf5(path)) 

schema = StructType([
  StructField("artist_familiarity", DoubleType(), True),
  StructField("artist_hotttnesss", DoubleType(), True),
  StructField("artist_id", StringType(), True),
  StructField("artist_latitude", DoubleType(), True),
  StructField("artist_location", StringType(), True),
  StructField("artist_longitude", DoubleType(), True),
  StructField("artist_name", StringType(), True),
  StructField("genre", StringType(), True),
  StructField("release", StringType(), True),
  StructField("song_hotttnesss", DoubleType(), True),
  StructField("song_id", StringType(), True),
  StructField("title", StringType(), True),
  StructField("danceability", DoubleType(), True),
  StructField("duration", DoubleType(), True),
  StructField("end_of_fade_in", DoubleType(), True),
  StructField("energy", DoubleType(), True),
  StructField("key", LongType(), True),
  StructField("loudness", DoubleType(), True),
  StructField("mode", LongType(), True),
  StructField("start_of_fade_out", DoubleType(), True),
  StructField("tempo", DoubleType(), True),
  StructField("time_signature", LongType(), True),
  StructField("track_id", StringType(), True),
  StructField("year", LongType(), True),
  StructField("artist_terms", ArrayType(StringType()), True),
  StructField("similar_artists", ArrayType(StringType()), True),
  StructField("bars_start", ArrayType(DoubleType()), True),
  StructField("beats_start", ArrayType(DoubleType()), True),
  StructField("sections_start", ArrayType(DoubleType()), True),
  StructField("segments_start", ArrayType(DoubleType()), True),
  StructField("tatums_start", ArrayType(DoubleType()), True)])

# Create dataframe
df = spark.createDataFrame(rdd,schema) 
print("NUM_SONGS_LOADED: ", df.count())
display(df)

In [2]:
# ETL --> TRANSFORM, add and remove some columns
from pyspark.sql.functions import col, size, create_map, lit, concat, element_at, when
import numpy as np
from itertools import chain

#df = spark.sql("SELECT * FROM SONGS")

# ADD COLUMNS: Density, Fadiness and Genre
df=df.withColumn('density', size(df.segments_start)/df.duration)
df=df.withColumn('fadiness', df.end_of_fade_in+df.duration-df.start_of_fade_out)
df=df.withColumn('variability', size(df.sections_start))

# ADD COLUMN: Tonality (Key + Mode)
dict_mode={1:'maj', 0:'min'}
dict_key ={0:'DO',1:'DO#',2:'RE',3:'RE#',4:'MI',5:'FA',6:'FA#',7:'SOL',8:'SOL#',9:'LA',10:'LA#',11:'SI'}

mapping_expr_key = create_map([lit(x) for x in chain(*dict_key.items())])
mapping_expr_mode = create_map([lit(x) for x in chain(*dict_mode.items())])

df=df.withColumn("key_string", (mapping_expr_key.getItem(col("key"))))
df=df.withColumn('mode_string', (mapping_expr_mode.getItem(col("mode"))))
df=df.withColumn('tonality', concat(df.key_string,lit(' '),df.mode_string))

# Set some value to 0 when there's NaN value 
df=df.withColumn("song_hotttnesss", when(col("song_hotttnesss")=="NaN", 0).otherwise(col("song_hotttnesss")))
df=df.withColumn("artist_familiarity", when(col("artist_familiarity")=="NaN", 0).otherwise(col("artist_familiarity")))

# Final dataframe
included =["artist_name","artist_familiarity","artist_hotttnesss","artist_id","artist_latitude","artist_longitude","artist_location", "release","song_hotttnesss","key_string","mode_string","tonality","title","duration","loudness","tempo","track_id","year","density","fadiness","variability","similar_artists"]
songs = df.select(included)

In [3]:
# ETL: Transfrom (Join with another table) - Improve genre lables with "Tagtraum genre annotation table"
df = spark.sql("SELECT SONGS.*, GENRE.SEED_GENRE AS GENRE FROM SONGS LEFT JOIN genre ON SONGS.track_id=genre.track_id")
display(df)

artist_name,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_longitude,artist_location,artist_genre,release,song_hotttnesss,key_string,mode_string,tonality,title,duration,loudness,tempo,track_id,year,density,fadiness,variability,similar_artists,GENRE
Natural Elements,0.4480205504320839,0.36953572756421016,ARPYBWQ1187B989C0D,NaN,NaN,,hip hop,Lyricist Lounge Vol. 1,0.40148671142756037,DO#,maj,DO# maj,Mayday,338.1024,-6.318,185.88,TRABDTV128E0791839,1998,4.28864154764947,4.333399999999983,10,"List(ARGWETC1187FB4EC41, AR8GJA41187B999863, ARSHO8N1187B9995F7, ARXU8YQ1187B999861, AREXDAJ1187FB38B4B, ARGQ1FN1187B998812, ARNIZ5P1187B989AF5, ARFH0111187B997931, ARHTK0R1187B98BF09, ARZ5V4M1187FB39063, ARDXUGZ11F4C84452F, ARIWGS71187B993991, AR22HWU1187FB5A9A6, ARLMHFV1187B9A3833, AR5PF241187B989C1D, ARHVZEM11F4C841FF9, ARQMWPF1187FB536C3, AR6M50Z1187B9A3503, ARJGOG11187B98D89F, ARH8A7P1187B98A805, ARE815V1187B998A43, AROJSPF1187B9B1FD1, ARF2OBW1187B9ACCDA, ARPF5261187B99DDA2, ART0OKB1187B9B8A2E, ARGTZL71187FB597C2, AR45W121187FB3D168, AROMXWS11F50C47E5D, ARUE7JM1187FB5A9AA, AR0D3LM1187B9B9592, ARDIP5S1187FB57056, ARP9H0U1187FB3FEA7, ARWFREB1187B98B09D, ARNH6Q11187FB40FDD, ARZD25L1187B98D5F5, ARAVZPG1187B98F815, ARCY77N1187B9ABEF9, AR123TC1187B98E3A3, ARPR9W71187FB3723A, ARWHS4W1187FB41E69, ARDB6821187B993ADE, ARB9E4C1187B9B12B6, ARD27C41187B98DD7D, ARQXANF11F4C84734E, AR5D2BB1187B9B1742, ARAV4ZE1187B9A1F60, ARLV7V21187FB3FF84, AR4UWS31187FB5C9A1, ARWHM281187FB3D381, ARGZNQM1269FCD2FC0, ARJDEQU1187B988C40, AR8D9AY1187B99F87F, AR5VBGP1187B98EB43, ARBEWHE11F50C512B2, ARHRRWZ11F4C846224, ARUGOBE1269FCD74CE, ARLQK3P1187FB42113, AR1O4JH1187FB3EE94, ARIKESW11F50C4F33C, AR5UGU21187FB42473, ARAQH7G1187FB59E30, ARYWRED11F50C47E83, ARN00NH1187B9ADE12, ARBYCLR1187FB50392, ARNQQJA1187B999862, ARMYH611187B9AE283, ARX77IM1187B98E616, AR9GUWJ1187B9A1ED1, ARCNGQZ1269FCD2FBF, ARRUXJG1187B98FD64, ARWBPEP11F50C47E6C, ARZ3TJ01187B989C6E, ARPZEFW11F50C47E67, ARGCOVG1269FB32D13, ARZBYR71187B9B3DBD, ARJG03W1187B9950C3, ARHSGAW1187B9898C7, AR89HJ01187B997B1B, ARUHUCE1269FCD5BDD, ARYYFT91187FB50113, ARKBKTR1187FB39255, ARMEMCV1187B989C13, ARWLCAW1269FB2EF64, ARWJRXP11F50C47E86, ARGTTRM11F50C47E69, ARN78J11187B98BA93, ARPKI8R1187B99A681, ARE8U941187B998678, ARKDU3G1187B994FE3, ARRVYKI1257509E4E2, ARDQW6Q1187B98E933, ARUNJO81187B9A8EFF, AR6PUI11187FB4069C, AROHTEU11F50C47E7E, ARJZDWQ11F50C47E6A, ARNP4ZJ1187B99EF2B, ARWXUPL11F50C47E7F, ARBPWDG1269FB32C56, ARRH2VW1187B991BB4, ARB4D891187B9954F7)",Hip-Hop
Kid Sister,0.7493950448979038,0.4645175256716385,ARJ9B1V1187FB4DD1F,NaN,NaN,Chicago IL,hip house,Ultraviolet,0.573019827988094,SI,min,SI min,Daydreaming [Jakwob Remix],277.2371,-3.573,140.026,TRABIOI12903CD8B9B,2009,3.7296595585511465,4.356100000000026,11,"List(ARB8BJE1187FB538D6, ARUYGXE12454A4D73F, AROB0091187FB47B7A, ARDVBYE1269FB26C31, ARUQ6301187FB54EBA, ARQ5LFQ11C8A4153FC, ARM6QJL1187B9A83CF, ARRH63Y1187FB47783, ARD46C811C8A414F3F, AR8L6F311C8A422086, ARDHKG01187FB4CBDE, ARFXHTN11F3462EBEB, ARRRRYQ12086C11272, AR47GV91187B996D90, ARYRDLR1187FB55CF5, ARWV5RG1187FB57129, ARFSMPX122BCFCA00D, ARS5ULQ1187B9AE810, ARTVNGD11C8A4227C9, ARIX1671187FB585C7, ARABR0A1187FB4D9CD, ARF4NL11187B9B9361, ARD91HZ1187FB3E8B7, ARCEWQV11F4C840563, ARZWCCS11F50C4E1A2, ARDYVEQ122BCFCC19E, ARHVZFT1242078074A, ARSLV4K11C8A415649, AR4HEGG1187B9A8654, AREGC1Z1187B995A90, ARWXLRK12454A3E429, ARDTDUT11F50C49068, ARZVDJU12592CDBBD3, ARMDZDX1269FCD3AD3, ARGRYAV122BCFCC810, AREL9YH119B866882B, ARV6L7F11C8A414EB6, AR0H4EH1187FB377CB, ARI3LFX1187FB5464D, AR1K7TQ11C8A414EC8, AR0UL1P1187FB4CEB3, ARXSEKI11F50C50318, ARUZ1U011C8A421449, AR7I7SX1187FB54297, ARLGOUD12298900ACE, AR0NZW11187FB4800E, ARVKJGO12086C15075, ARUWBM11187FB38F16, ARVGWT4119B8668115, AR4YSCE1187B9AF1F9, ARIGPNM1187B98C744, ARSFK4D1187B9A8F08, ARF5UVP1187B9AE715, ARDDMYP1187B9B8AE4, ARE65ST1187FB4F994, ARFSQKP1187FB50B59, ARHAEXZ1187FB54F3C, AR3MSAN1187B98F06B, ARNSY131187B991F73, AR3Q2FQ1187FB5CAFE, ARDVPN11

In [4]:
# ETL --> LOAD and save dataframe as HIVE TABLE
df.write.mode('overwrite').format("parquet").saveAsTable("songs")
display(df)

artist_name,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_longitude,artist_location,artist_genre,release,song_hotttnesss,key_string,mode_string,tonality,title,duration,loudness,tempo,track_id,year,density,fadiness,variability,similar_artists,GENRE
Natural Elements,0.4480205504320839,0.36953572756421016,ARPYBWQ1187B989C0D,NaN,NaN,,hip hop,Lyricist Lounge Vol. 1,0.40148671142756037,DO#,maj,DO# maj,Mayday,338.1024,-6.318,185.88,TRABDTV128E0791839,1998,4.28864154764947,4.333399999999983,10,"List(ARGWETC1187FB4EC41, AR8GJA41187B999863, ARSHO8N1187B9995F7, ARXU8YQ1187B999861, AREXDAJ1187FB38B4B, ARGQ1FN1187B998812, ARNIZ5P1187B989AF5, ARFH0111187B997931, ARHTK0R1187B98BF09, ARZ5V4M1187FB39063, ARDXUGZ11F4C84452F, ARIWGS71187B993991, AR22HWU1187FB5A9A6, ARLMHFV1187B9A3833, AR5PF241187B989C1D, ARHVZEM11F4C841FF9, ARQMWPF1187FB536C3, AR6M50Z1187B9A3503, ARJGOG11187B98D89F, ARH8A7P1187B98A805, ARE815V1187B998A43, AROJSPF1187B9B1FD1, ARF2OBW1187B9ACCDA, ARPF5261187B99DDA2, ART0OKB1187B9B8A2E, ARGTZL71187FB597C2, AR45W121187FB3D168, AROMXWS11F50C47E5D, ARUE7JM1187FB5A9AA, AR0D3LM1187B9B9592, ARDIP5S1187FB57056, ARP9H0U1187FB3FEA7, ARWFREB1187B98B09D, ARNH6Q11187FB40FDD, ARZD25L1187B98D5F5, ARAVZPG1187B98F815, ARCY77N1187B9ABEF9, AR123TC1187B98E3A3, ARPR9W71187FB3723A, ARWHS4W1187FB41E69, ARDB6821187B993ADE, ARB9E4C1187B9B12B6, ARD27C41187B98DD7D, ARQXANF11F4C84734E, AR5D2BB1187B9B1742, ARAV4ZE1187B9A1F60, ARLV7V21187FB3FF84, AR4UWS31187FB5C9A1, ARWHM281187FB3D381, ARGZNQM1269FCD2FC0, ARJDEQU1187B988C40, AR8D9AY1187B99F87F, AR5VBGP1187B98EB43, ARBEWHE11F50C512B2, ARHRRWZ11F4C846224, ARUGOBE1269FCD74CE, ARLQK3P1187FB42113, AR1O4JH1187FB3EE94, ARIKESW11F50C4F33C, AR5UGU21187FB42473, ARAQH7G1187FB59E30, ARYWRED11F50C47E83, ARN00NH1187B9ADE12, ARBYCLR1187FB50392, ARNQQJA1187B999862, ARMYH611187B9AE283, ARX77IM1187B98E616, AR9GUWJ1187B9A1ED1, ARCNGQZ1269FCD2FBF, ARRUXJG1187B98FD64, ARWBPEP11F50C47E6C, ARZ3TJ01187B989C6E, ARPZEFW11F50C47E67, ARGCOVG1269FB32D13, ARZBYR71187B9B3DBD, ARJG03W1187B9950C3, ARHSGAW1187B9898C7, AR89HJ01187B997B1B, ARUHUCE1269FCD5BDD, ARYYFT91187FB50113, ARKBKTR1187FB39255, ARMEMCV1187B989C13, ARWLCAW1269FB2EF64, ARWJRXP11F50C47E86, ARGTTRM11F50C47E69, ARN78J11187B98BA93, ARPKI8R1187B99A681, ARE8U941187B998678, ARKDU3G1187B994FE3, ARRVYKI1257509E4E2, ARDQW6Q1187B98E933, ARUNJO81187B9A8EFF, AR6PUI11187FB4069C, AROHTEU11F50C47E7E, ARJZDWQ11F50C47E6A, ARNP4ZJ1187B99EF2B, ARWXUPL11F50C47E7F, ARBPWDG1269FB32C56, ARRH2VW1187B991BB4, ARB4D891187B9954F7)",Hip-Hop
Kid Sister,0.7493950448979038,0.4645175256716385,ARJ9B1V1187FB4DD1F,NaN,NaN,Chicago IL,hip house,Ultraviolet,0.573019827988094,SI,min,SI min,Daydreaming [Jakwob Remix],277.2371,-3.573,140.026,TRABIOI12903CD8B9B,2009,3.7296595585511465,4.356100000000026,11,"List(ARB8BJE1187FB538D6, ARUYGXE12454A4D73F, AROB0091187FB47B7A, ARDVBYE1269FB26C31, ARUQ6301187FB54EBA, ARQ5LFQ11C8A4153FC, ARM6QJL1187B9A83CF, ARRH63Y1187FB47783, ARD46C811C8A414F3F, AR8L6F311C8A422086, ARDHKG01187FB4CBDE, ARFXHTN11F3462EBEB, ARRRRYQ12086C11272, AR47GV91187B996D90, ARYRDLR1187FB55CF5, ARWV5RG1187FB57129, ARFSMPX122BCFCA00D, ARS5ULQ1187B9AE810, ARTVNGD11C8A4227C9, ARIX1671187FB585C7, ARABR0A1187FB4D9CD, ARF4NL11187B9B9361, ARD91HZ1187FB3E8B7, ARCEWQV11F4C840563, ARZWCCS11F50C4E1A2, ARDYVEQ122BCFCC19E, ARHVZFT1242078074A, ARSLV4K11C8A415649, AR4HEGG1187B9A8654, AREGC1Z1187B995A90, ARWXLRK12454A3E429, ARDTDUT11F50C49068, ARZVDJU12592CDBBD3, ARMDZDX1269FCD3AD3, ARGRYAV122BCFCC810, AREL9YH119B866882B, ARV6L7F11C8A414EB6, AR0H4EH1187FB377CB, ARI3LFX1187FB5464D, AR1K7TQ11C8A414EC8, AR0UL1P1187FB4CEB3, ARXSEKI11F50C50318, ARUZ1U011C8A421449, AR7I7SX1187FB54297, ARLGOUD12298900ACE, AR0NZW11187FB4800E, ARVKJGO12086C15075, ARUWBM11187FB38F16, ARVGWT4119B8668115, AR4YSCE1187B9AF1F9, ARIGPNM1187B98C744, ARSFK4D1187B9A8F08, ARF5UVP1187B9AE715, ARDDMYP1187B9B8AE4, ARE65ST1187FB4F994, ARFSQKP1187FB50B59, ARHAEXZ1187FB54F3C, AR3MSAN1187B98F06B, ARNSY131187B991F73, AR3Q2FQ1187FB5CAFE, ARDVPN11